In [9]:
import requests,urllib
import pandas as pd

In [10]:
def get_json(base_url,params):
    params_str=urllib.parse.urlencode(params)
    url=base_url+params_str
    json=requests.get(url).json()
    return json

In [11]:
appID="1228171589a8c7453c27fced05d8d1e1fa944be8"
base_url="http://api.e-stat.go.jp/rest/3.0/app/json/getStatsData?"
params={
    "appId":appID,
    "lang":"J",
    "statsDataId":"0003171820",
    "metaGetFlg":"Y",
    "cntGetFlg":"N",
    "explanationGetFlg":"Y",
    "annotationGetFlg":"Y",
    "sectionHeaderFlg":"1"}

In [12]:
#http://api.e-stat.go.jp/rest/3.0/app/json/getStatsData?appId=&lang=J&statsDataId=0003171820&metaGetFlg=Y&cntGetFlg=N&explanationGetFlg=Y&annotationGetFlg=Y&sectionHeaderFlg=1

In [13]:
json=get_json(base_url,params)

In [14]:
df_original=pd.DataFrame(json["GET_STATS_DATA"]["STATISTICAL_DATA"]["DATA_INF"]["VALUE"])

In [15]:
df_original

,@tab,@cat01,@cat03,@time,@unit,$
0,2850,1,110,9,人,5.00
1,2850,1,120,9,人,4.79
2,2850,1,130,9,人,4.68
3,2850,1,140,9,人,4.53
4,2850,1,150,9,人,4.38
...,...,...,...,...,...,...
184,2850,3,690,9,人,2.61
185,2850,3,700,9,人,2.60
186,2850,3,710,9,人,2.54
187,2850,3,720,9,人,2.53


In [16]:
# select data on average (all) households
df_average_hh = df_original.loc[df_original['@cat01'] == '1']

In [17]:
df_average_hh

,@tab,@cat01,@cat03,@time,@unit,$
0,2850,1,110,9,人,5.00
1,2850,1,120,9,人,4.79
2,2850,1,130,9,人,4.68
3,2850,1,140,9,人,4.53
4,2850,1,150,9,人,4.38
...,...,...,...,...,...,...
58,2850,1,690,9,人,2.58
59,2850,1,700,9,人,2.57
60,2850,1,710,9,人,2.51
61,2850,1,720,9,人,2.49


In [18]:
df_average_hh.columns

Index(['@tab', '@cat01', '@cat03', '@time', '@unit', '$'], dtype='object')

In [19]:
# get time df
class_data=json["GET_STATS_DATA"]["STATISTICAL_DATA"]["CLASS_INF"]["CLASS_OBJ"]
time_df=pd.DataFrame(class_data[2]["CLASS"])

In [20]:
# get year in western calender
time_df['year_wst'] = time_df['@name'].str[-6:-2]

In [21]:
time_df

,@code,@name,@level,year_wst
0,110,昭和28年(1953年),1,1953
1,120,昭和29年(1954年),1,1954
2,130,昭和30年(1955年),1,1955
3,140,昭和31年(1956年),1,1956
4,150,昭和32年(1957年),1,1957
...,...,...,...,...
58,690,平成23年(2011年),1,2011
59,700,平成24年(2012年),1,2012
60,710,平成25年(2013年),1,2013
61,720,平成26年(2014年),1,2014


In [22]:
time_df = time_df.loc[:, ['@code', 'year_wst']]

In [23]:
df_merged = pd.merge(df_average_hh, time_df, left_on = '@cat03', right_on = '@code')

In [24]:
df_merged = df_merged.loc[:, ["$", "year_wst"]]

In [25]:
df_merged.columns = ['hh_size', 'year_wst']

In [26]:
df_merged.to_csv("../../Data/Downloaded/post_hh_size.csv", index = False)